In [ ]:
import omicverse as ov
import scanpy as sc
import pandas as pd
import scvi
import scib
from matplotlib.pyplot import rc_context

In [ ]:
print(f'omicverse version: {ov.__version__}')
print(f'scanpy version: {sc.__version__}')

In [ ]:
import torch # 如果pytorch安装成功即可导入
print(torch.cuda.is_available()) # 查看CUDA是否可用
print(torch.cuda.device_count()) # 查看可用的CUDA数量
print(torch.version.cuda) # 查看CUDA的版本号
print(torch.cuda.device_count())

In [ ]:
from Function.clean_adata import *
from Function.remove_noise_genes import *
from Function.save_embeding import *

In [ ]:
adata=sc.read_h5ad('/data/data_mailab003/project/scrna-npc-22-tl/Downstream/integrate/process_data/seu_preprocess_shiftlog_pearson_hvg2000_pc20_louvain_res1.0_add_group.h5ad')
adata

In [ ]:
sc.settings.figdir = "/data/data_mailab003/project/scrna-npc-22-tl/Downstream/results/batch"

In [ ]:
with rc_context({'figure.figsize': (8, 8)}):
    sc.pl.umap(adata,color=["louvain_res1.0","major_celltype"],frameon=False, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2,save="before_batch.png")

In [ ]:
## note: the newest version of omicverse modified the code of batch correction, currently it should be : ov.single.batch_correction(adata,batch_key='study',methods='harmony',n_pcs=50)
adata=ov.single.batch_correction(adata,batch_key='study',methods='harmony',n_pcs=50)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=20, use_rep='X_pca_harmony',key_added="neighbors_harmony")
sc.tl.umap(adata,neighbors_key="neighbors_harmony")
adata.obsm['X_umap_harmony']=adata.obsm['X_umap']

In [ ]:
ov.utils.cluster(adata,method="louvain",key_added="louvain_res1.0", neighbors_key="neighbors_harmony",resolution=1.0)

In [ ]:
with rc_context({'figure.figsize': (8, 8)}):
    sc.pl.umap(adata,color=["louvain_res1.0","major_celltype","study"],neighbors_key="neighbors_harmony",frameon=False, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2)

In [ ]:
adata.write_h5ad("/data/data_mailab003/project/scrna-npc-22-tl/Downstream/integrate/process_data/seu_preprocess_shiftlog_pearson_hvg2000_pc20_louvain_res1.0_add_group_harmony.h5ad")